In [1]:
import pandas as pd
import numpy as np
from numpy import absolute
from numpy import mean
from numpy import std
from sklearn import preprocessing, metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [2]:
df01 = pd.read_csv("bellman_performance_results_final.csv")
df02 = pd.read_csv("dijkstra_performance_results_final.csv")
df03 = pd.read_csv("hybrid_performance_results_final.csv")

In [3]:
df01['AlgoBellman'] = int(10)
df01['AlgoDijkstra'] = int(0)
df01['AlgoHybrid'] = int(0)
df02['AlgoBellman'] = int(0)
df02['AlgoDijkstra'] = int(10)
df02['AlgoHybrid'] = int(0)
df03['AlgoBellman'] = int(0)
df03['AlgoDijkstra'] = int(0)
df03['AlgoHybrid'] = int(10)

In [4]:
frames = [df01, df02, df03]
concat = pd.concat(frames)
df1 = concat

In [5]:
conditions = [df1['vertexNum'] == 524288, df1['vertexNum'] == 1048576, df1['vertexNum'] == 2097152, df1['vertexNum'] == 4194304, df1['vertexNum'] == 8388608, df1['vertexNum'] == 16777216, df1['vertexNum'] == 33554432]
numbers = [1, 2, 4, 8, 16, 32, 64]
df1['graphSize'] = np.select(conditions, numbers, default=0)

In [6]:
#one-hot encoding - nothing is dropped

df1 = df1.drop(['vertexNum', 'edgeNum', 'maxEdgeDegree',
                'sAtomicMinBlock', 'sAtomicMaxBlock',
                'sAtomicAddBlock', 'sAtomicExchBlock'], axis=1)

df1.dropna()


,iterationNum,minProcessEdge,percentage,sOriginalDistance,sMinEdgetoProcess,sApprAttrValues,sReduceExecution,sPartialGraphProcess,Error,executionTime,AlgoBellman,AlgoDijkstra,AlgoHybrid,graphSize
0,7,0,1.0,1,0,0,0,0,0.000000,26.250,10,0,0,1
1,7,2,1.0,0,1,0,0,0,0.151353,25.125,10,0,0,1
2,7,3,1.0,0,1,0,0,0,0.255608,24.500,10,0,0,1
3,7,4,1.0,0,1,0,0,0,0.376860,24.000,10,0,0,1
4,7,6,1.0,0,1,0,0,0,0.611287,24.000,10,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,3,0,1.0,0,0,0,1,0,0.000017,1501.000,0,0,10,64
183,4,0,1.0,0,0,0,1,0,0.000021,1487.875,0,0,10,64
184,5,0,1.0,0,0,0,1,0,0.000020,1492.000,0,0,10,64
185,6,0,1.0,0,0,0,1,0,0.000016,1521.875,0,0,10,64


In [7]:
df1 = df1[df1['minProcessEdge'] > 0]



for i in numbers:
    mini = df1[df1['graphSize'] == i]['minProcessEdge'].min()
    maxi = df1[df1['graphSize'] == i]['minProcessEdge'].max()
    #print(mini, maxi, i)
    df1.loc[df1['graphSize'] == i, ['minProcessEdge']] = 1.0 / (((df1.loc[df1['graphSize'] == i, ['minProcessEdge']] - mini) / (maxi - mini)) * 9 + 1)



In [8]:
print(df1.head())


   iterationNum  minProcessEdge  percentage  sOriginalDistance  \
1             7        1.000000         1.0                  0   
2             7        0.979955         1.0                  0   
3             7        0.960699         1.0                  0   
4             7        0.924370         1.0                  0   
5             7        0.844530         1.0                  0   

   sMinEdgetoProcess  sApprAttrValues  sReduceExecution  sPartialGraphProcess  \
1                  1                0                 0                     0   
2                  1                0                 0                     0   
3                  1                0                 0                     0   
4                  1                0                 0                     0   
5                  1                0                 0                     0   

      Error  executionTime  AlgoBellman  AlgoDijkstra  AlgoHybrid  graphSize  
1  0.151353         25.125           

In [9]:
#################### EXECUTION TIME #############

In [10]:
X = df1.drop(['executionTime', 'Error'],axis=1)
y = df1[['executionTime']]

In [11]:
#print(X)
#print(type(X))
X = X.values

#print(type(X))
#print(type(y))
#y = y.values
#print(type(y))
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)


In [12]:
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

#print(model.intercept_)

print(r2_score(y_pred, y_test))

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.9018582295624333
Mean Absolute Error: 89.91936604988265
Mean Squared Error: 14974.230375937628
Root Mean Squared Error: 122.3692378661305


In [13]:
print(y_test)
print(y_pred)
#print(np.exp(y_test))
#print(np.exp(y_pred))

     executionTime
1           40.000
185        767.500
123        241.375
61          75.875
34          61.875
..             ...
122        362.625
164       1450.500
148        736.500
32          64.500
194        606.000

[101 rows x 1 columns]
[[ 151.50707062]
 [1052.6827978 ]
 [ 222.44625091]
 [  14.56703115]
 [ 138.78586164]
 [  52.47405001]
 [-157.9130105 ]
 [ 133.90755011]
 [ 259.83783239]
 [ -32.55151645]
 [ 411.31474109]
 [ 509.73653309]
 [-107.32853314]
 [1209.64151236]
 [ 684.61984265]
 [ 263.69484996]
 [  81.4011259 ]
 [ 147.50933302]
 [  43.3332667 ]
 [ 189.48160521]
 [1203.72627853]
 [  74.82635978]
 [ 268.20441179]
 [ 684.51005435]
 [ 120.49966635]
 [ 186.29446873]
 [ 143.66871175]
 [1047.49123918]
 [ 268.25935297]
 [ 260.85140055]
 [  83.6766284 ]
 [ 126.81575512]
 [ 683.96292146]
 [ 138.62666123]
 [  45.11798815]
 [1055.11197132]
 [1055.3855885 ]
 [ 682.90296808]
 [ 494.3440972 ]
 [ 198.21949202]
 [ 269.7354769 ]
 [ 201.15873832]
 [  50.26157105]
 [ 686.89957084]


In [14]:
'''model = KNeighborsRegressor()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

#print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
#print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
#print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))'''

"model = KNeighborsRegressor()\nmodel.fit(x_train, y_train)\ny_pred = model.predict(x_test)\n\n#print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))\n#print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))\n#print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))"

In [15]:
'''print(y_test)
print(y_pred)'''

'print(y_test)\nprint(y_pred)'

In [16]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2 score: ', r2_score(y_test, y_pred))

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Absolute Error: 6.721534653465347
Mean Squared Error: 171.20436262376236
Root Mean Squared Error: 13.084508497599838
R2 score:  0.9988961258381761
MAE: 9.055 (7.491)


In [17]:
print(y_test)
print(y_pred)

     executionTime
1           40.000
185        767.500
123        241.375
61          75.875
34          61.875
..             ...
122        362.625
164       1450.500
148        736.500
32          64.500
194        606.000

[101 rows x 1 columns]
[  40.     759.625  245.      76.5     64.375   39.      21.      64.
  219.25    36.     436.     391.75    22.    1305.375  808.625  215.625
  129.875   40.     193.625  193.625 1289.5    124.375  246.     819.125
   39.875  118.      40.     654.75   246.     222.25   144.      61.375
  756.125   66.      38.125  909.25   928.375  716.375  368.625  127.125
  226.     126.125   38.125  767.375 1478.75    43.      37.     250.
   39.     215.625 1405.375  388.     368.625   64.     246.      18.875
  126.375 1238.125  449.875  304.375  231.125  124.125  117.875   64.5
  128.625  119.125   74.875  426.25   814.375  131.75    76.5    718.
   22.     654.75    39.875   37.     119.875   24.5     36.875  391.
  215.625  449.875  245.     129

In [18]:
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(x_train)
X_testscaled=sc_X.transform(x_test)
#X_trainscaled = x_train
#X_testscaled = x_test
#print(X_trainscaled)

In [19]:
#t = [(2),(3),(4),(5),(6),(7),(8),(9),(10),(2,2),(3,2),(4,2),(5,2),(6,2),(7,2),(8,2)]

model = MLPRegressor(hidden_layer_sizes=(4,8,4),activation="relu" ,random_state=1, max_iter=50000)
model.fit(X_trainscaled, y_train.values.ravel()).loss_
print(model.loss_)
#print(model.loss_curve_)
print(model.n_outputs_)
print(model.best_loss_)
y_pred=model.predict(X_testscaled)


print(r2_score(y_pred, y_test))
print("Training set score: %f" % model.score(X_trainscaled, y_train.values.ravel()))
print("Test set score: %f" % model.score(X_testscaled, y_test))

578.4827661505143
1
577.4803249241934
0.9889575558945216
Training set score: 0.992682
Test set score: 0.989163


In [20]:
print(y_test)
print(y_pred)
print("Test set score: %f" % model.score(X_testscaled, y_test))

     executionTime
1           40.000
185        767.500
123        241.375
61          75.875
34          61.875
..             ...
122        362.625
164       1450.500
148        736.500
32          64.500
194        606.000

[101 rows x 1 columns]
[  92.60435913  779.45795782  253.98402675   67.42067949   67.78394772
   43.73159831   58.35612033   65.09027015  211.20377553   33.95426036
  405.3345021   351.365158     14.36329575 1356.20066892  786.64177499
  217.41570214  126.26530891   90.48641466   58.35612033  209.16628897
 1347.09177737  117.32645525  215.63627019  786.1961592    76.69837442
  139.05245082   88.45170819  758.38612629  215.66537725  211.74074951
  129.35901347   49.88596678  783.97542107   67.36958555   40.04569057
  789.31764335  790.4282183   764.77297484  404.33164214  141.20162479
  220.61594031  146.92732925   42.62298973  780.99464281 1388.46524229
   37.3465371    27.05206787  254.43051037   41.21808649  248.88167494
 1376.83987288  386.45579958  402.4441

In [21]:
print("y1 MSE:%.4f" % mean_squared_error(y_test, y_pred)) 

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

#print("y2 MSE:%.4f" % mean_squared_error(y_test[:,1], y_pred[:,1]))

y1 MSE:1680.7304
Mean Absolute Error: 27.72934183476286
Mean Squared Error: 1680.7303804050305
Root Mean Squared Error: 40.996711824304036


# ERROR #

In [22]:
df2 = concat

In [23]:
#one-hot encoding - nothing is dropped

df2 = df2.drop(['vertexNum', 'edgeNum', 'maxEdgeDegree',
                'sAtomicMinBlock', 'sAtomicMaxBlock',
                'sAtomicAddBlock', 'sAtomicExchBlock'], axis=1)
df2.dropna()

print(df2.head(20))

    iterationNum  minProcessEdge  percentage  sOriginalDistance  \
0              7               0         1.0                  1   
1              7               2         1.0                  0   
2              7               3         1.0                  0   
3              7               4         1.0                  0   
4              7               6         1.0                  0   
5              7              11         1.0                  0   
6              7              17         1.0                  0   
7              7              46         1.0                  0   
8              7              48         1.0                  0   
9              7              50         1.0                  0   
10             7              53         1.0                  0   
11             7              60         1.0                  0   
12             7             132         1.0                  0   
13             7             140         1.0                  

In [24]:
maxi = df2['minProcessEdge'].max()
df2['minProcessEdge'] /= maxi; 
maxi = df2['iterationNum'].max()
mini = df2['iterationNum'].min()
#df2['iterationNum'] = 1.0 / (((df2['minProcessEdge'] - mini) / (maxi - mini)) * 9 + 1)
df2['iterationNum'] = 1 / (df2['iterationNum'] + 1)

In [25]:
print(df2.head(20))

    iterationNum  minProcessEdge  percentage  sOriginalDistance  \
0          0.125        0.000000         1.0                  1   
1          0.125        0.000061         1.0                  0   
2          0.125        0.000092         1.0                  0   
3          0.125        0.000122         1.0                  0   
4          0.125        0.000183         1.0                  0   
5          0.125        0.000336         1.0                  0   
6          0.125        0.000519         1.0                  0   
7          0.125        0.001405         1.0                  0   
8          0.125        0.001466         1.0                  0   
9          0.125        0.001527         1.0                  0   
10         0.125        0.001619         1.0                  0   
11         0.125        0.001833         1.0                  0   
12         0.125        0.004032         1.0                  0   
13         0.125        0.004276         1.0                  

In [26]:
X = df2.drop(['executionTime', 'Error'],axis=1)
y = df2[['Error']]


In [27]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [28]:
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

#print(model.intercept_)

print(r2_score(y_pred, y_test))

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.9152053741494097
Mean Absolute Error: 2.7624300672910556
Mean Squared Error: 12.939184304971846
Root Mean Squared Error: 3.5971077694408664


In [29]:
print(y_test)
print(y_pred)

         Error
195  40.221508
1     0.153689
15   13.730715
77    0.000653
51    0.001183
83    0.000000
114  10.619989
23    0.003720
184  50.000000
29    0.000000
81    0.120040
153   8.841710
125   2.679620
125   9.914795
77    0.000470
31    0.565010
188   0.008912
1     0.154244
133   9.914734
165   0.000063
133   0.000204
99    4.782403
60    1.480589
51    0.000757
39    5.745185
135  49.436127
83    0.398604
71   31.421837
155   2.627121
139   0.000000
0     0.000000
93    0.693523
157   3.044508
134   0.000000
115   3.602237
144   8.670356
95    7.053701
3     0.376860
177   0.000000
85   11.073553
44   13.173957
64    4.793233
69    5.662141
37    5.745683
33    1.836053
140   2.627171
118   0.000000
36    5.746547
33    0.564119
139   1.123727
197  42.458393
108  15.613996
22    0.003103
144   4.141856
181   7.662958
88    4.074815
13    9.757296
6     2.093466
40   10.311871
[[ 4.78750215e+01]
 [ 5.89559002e+00]
 [ 4.14611438e+00]
 [ 2.21795037e+00]
 [ 5.96010991e-01]
 [ 9.

In [30]:
model = KNeighborsRegressor()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 5.936697559322034
Mean Squared Error: 155.91907112216478
Root Mean Squared Error: 12.48675582856351


In [31]:
print(y_test)
print(y_pred)

         Error
195  40.221508
1     0.153689
15   13.730715
77    0.000653
51    0.001183
83    0.000000
114  10.619989
23    0.003720
184  50.000000
29    0.000000
81    0.120040
153   8.841710
125   2.679620
125   9.914795
77    0.000470
31    0.565010
188   0.008912
1     0.154244
133   9.914734
165   0.000063
133   0.000204
99    4.782403
60    1.480589
51    0.000757
39    5.745185
135  49.436127
83    0.398604
71   31.421837
155   2.627121
139   0.000000
0     0.000000
93    0.693523
157   3.044508
134   0.000000
115   3.602237
144   8.670356
95    7.053701
3     0.376860
177   0.000000
85   11.073553
44   13.173957
64    4.793233
69    5.662141
37    5.745683
33    1.836053
140   2.627171
118   0.000000
36    5.746547
33    0.564119
139   1.123727
197  42.458393
108  15.613996
22    0.003103
144   4.141856
181   7.662958
88    4.074815
13    9.757296
6     2.093466
40   10.311871
[[6.97835760e+00]
 [1.39125460e+00]
 [8.37047440e+00]
 [3.22700000e-03]
 [2.39134000e-01]
 [1.101412

In [32]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2 score: ', r2_score(y_test, y_pred))

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performanceNN
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Absolute Error: 0.20554506779661016
Mean Squared Error: 0.39331729878859345
Root Mean Squared Error: 0.627150140547376
R2 score:  0.9971715446798024
MAE: 0.136 (0.086)


In [33]:
print(y_test)
print(y_pred)

         Error
195  40.221508
1     0.153689
15   13.730715
77    0.000653
51    0.001183
83    0.000000
114  10.619989
23    0.003720
184  50.000000
29    0.000000
81    0.120040
153   8.841710
125   2.679620
125   9.914795
77    0.000470
31    0.565010
188   0.008912
1     0.154244
133   9.914734
165   0.000063
133   0.000204
99    4.782403
60    1.480589
51    0.000757
39    5.745185
135  49.436127
83    0.398604
71   31.421837
155   2.627121
139   0.000000
0     0.000000
93    0.693523
157   3.044508
134   0.000000
115   3.602237
144   8.670356
95    7.053701
3     0.376860
177   0.000000
85   11.073553
44   13.173957
64    4.793233
69    5.662141
37    5.745683
33    1.836053
140   2.627171
118   0.000000
36    5.746547
33    0.564119
139   1.123727
197  42.458393
108  15.613996
22    0.003103
144   4.141856
181   7.662958
88    4.074815
13    9.757296
6     2.093466
40   10.311871
[4.0221981e+01 1.5135300e-01 1.3728124e+01 5.9400000e-04 1.3140000e-03
 0.0000000e+00 8.6938700e+00 

In [34]:
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(x_train)
X_testscaled=sc_X.transform(x_test)

In [35]:
model = MLPRegressor(hidden_layer_sizes=(4,8,4),activation="relu" ,random_state=1, max_iter=50000)
model.fit(X_trainscaled, y_train.values.ravel()).loss_
print(model.loss_)
#print(model.loss_curve_)
print(model.n_outputs_)
print(model.best_loss_)
y_pred=model.predict(X_testscaled)


print(r2_score(y_pred, y_test))
print("Training set score: %f" % model.score(X_trainscaled, y_train.values.ravel()))
print("Test set score: %f" % model.score(X_testscaled, y_test))

2.24553905972932
1
2.244299856539221
0.963067246139957
Training set score: 0.979779
Test set score: 0.959509


In [36]:
print(y_test)
print(y_pred)
print("Test set score: %f" % model.score(X_testscaled, y_test))

         Error
195  40.221508
1     0.153689
15   13.730715
77    0.000653
51    0.001183
83    0.000000
114  10.619989
23    0.003720
184  50.000000
29    0.000000
81    0.120040
153   8.841710
125   2.679620
125   9.914795
77    0.000470
31    0.565010
188   0.008912
1     0.154244
133   9.914734
165   0.000063
133   0.000204
99    4.782403
60    1.480589
51    0.000757
39    5.745185
135  49.436127
83    0.398604
71   31.421837
155   2.627121
139   0.000000
0     0.000000
93    0.693523
157   3.044508
134   0.000000
115   3.602237
144   8.670356
95    7.053701
3     0.376860
177   0.000000
85   11.073553
44   13.173957
64    4.793233
69    5.662141
37    5.745683
33    1.836053
140   2.627171
118   0.000000
36    5.746547
33    0.564119
139   1.123727
197  42.458393
108  15.613996
22    0.003103
144   4.141856
181   7.662958
88    4.074815
13    9.757296
6     2.093466
40   10.311871
[ 4.83865129e+01  3.30716731e+00  7.00872615e+00  2.33004030e-01
  2.33004030e-01  7.76529958e-01  1

In [37]:
print("y1 MSE:%.4f" % mean_squared_error(y_test, y_pred)) 

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

y1 MSE:5.6306
Mean Absolute Error: 1.5670521204852406
Mean Squared Error: 5.630633730612474
Root Mean Squared Error: 2.3728956425878645
